In [ ]:
from dask_jobqueue import SLURMCluster
from distributed import Client
cluster = SLURMCluster()
print(cluster.job_script())
client = Client(cluster)
client

In [ ]:
%%time
import pandas as pd
import fastparquet as fp
import dask.dataframe as dd

# Scale up before running compute
cluster.scale(jobs=4)

FN = '/gpfs/alpine/stf218/proj-shared/stf008stc/openbmc.summit.raw/openbmc-202004*-*.parquet'
df = dd.read_parquet(
    FN,
    engine='fastparquet',
    index=False,
    columns=['timestamp', 'total_power'],
    gather_statistics=False,
).repartition(partition_size="100MB").set_index('timestamp')
df = client.persist(df)
df

In [ ]:
%%time
value = df['total_power'].std().compute()
value

In [ ]:
client.close()
cluster.close()

In [ ]:
cluster.scale(jobs=0)